In [31]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

In [32]:
train_data = pd.read_csv("data/train.csv")

In [36]:
train_data

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,TweetID,mentions_count
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],1646978048000,832509,0.0
1,populaire,0,0,86,1016,284,[],[],0,[],1647694288000,1388011,0.0
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],1647370048000,63896,0.0
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],1647256282000,979251,0.0
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],1647258374000,1040049,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,gonflette tour raciste frustré,0,0,34,1509,55,[],['https://t.co/pma33zhslx'],0,[],1647438153000,142573,0.0
353965,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,[],[],0,[],1647072106000,240866,0.0
353966,eric zemmour français berbère,3,0,1888,712,3086,[],[],0,[],1647607230000,1173763,0.0
353967,gauchistes dépression pq,0,0,139,486,320,[],[],0,[],1646987195000,929182,0.0


In [35]:
train_data.sort_values('mentions_count')

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,TweetID,mentions_count
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],1646978048000,832509,0.0
235982,payet gauche match,0,0,155,1067,524,[],[],0,[],1646936797000,658189,0.0
235981,système rappelle totalement besoin changé,0,0,97,3671,387,[],[],0,[],1646953982000,847923,0.0
235980,preuve faciste,0,0,126,381,98,[],[],0,[],1647546796000,486558,0.0
235979,étonnant classement crédible fin tour hidalgo ...,0,0,848,8493,981,[],['https://t.co/sddtvkb0pr'],0,[],1647679279000,1330855,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117986,point revalorisé prochain traduction allez fon...,4,0,787,14534,794,[],['https://t.co/susgzdhtxa'],0,[],1647278153000,1000688,0.0
117985,rt bonjour préciser information exclusive rmc ...,0,0,274,1387,585,[],[],0,[],1646986416000,937617,0.0
117984,rt cinglant propos président sénat di,0,0,132,32548,81,[],[],0,['presidentielle2022'],1647581845000,1286083,0.0
117992,rt étoffer macron élu macron so,1,0,622,178592,873,[],[],0,[],1647599014000,1181039,0.0


In [ ]:
# new features
train_data["url_count"] = train_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["text_len"] = train_data["text"].apply(lambda s: len(s))
train_data["hashtags_count"] = train_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["day"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
train_data["hour"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)

# indicators of keywords
train_data["Macron"] =  train_data["text"].apply(lambda s: ("macron" in s.lower().split()))
train_data["Zemmour"] =  train_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
train_data["Melenchon"] =  train_data["text"].apply(lambda s: ("melenchon" in s.replace("é","e").lower().split()))
train_data["rt"] =  train_data["text"].apply(lambda s: ("rt" in s.lower().split()))

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
print("sentiment analysis...")
train_data["compound"] =  train_data["text"].apply(lambda s: sia.polarity_scores(s)['compound'])

In [ ]:
display(train_data.sort_values("compound")[["text","compound"]])

In [ ]:
train_data["avg_word_len"] = train_data["text"].apply(lambda s: np.mean([len(w) for w in s.split()]))
train_data["rep_words_freq"] = train_data["text"].apply(lambda s: np.mean(len(list(set(s.split())))/len(s.split())))

In [ ]:
train_data[["avg_word_len", "rep_words_freq", "favorites_count", "retweets_count"]].sort_values("avg_word_len")

In [24]:
train_data["rep_chars_freq"] = train_data["text"].apply(lambda s: np.mean(len(list(set(s)))/len(list(s))))

In [25]:
train_data.sort_values("rep_chars_freq")

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,day,hour,Macron,Zemmour,Melenchon,rt,compound,avg_word_len,rep_words_freq,rep_chars_freq
134183,vote vote vote vote vote vote vote vote vote v...,0,0,164,31083,80,[],['https://t.co/04wft89icf'],0,[],...,18,21,False,False,False,False,0.0,4.000000,0.025000,0.025126
102001,vote vote vote vote vote vote vote vote vote v...,9,11,363,5151,333,[],['https://t.co/w88mzuszit'],0,[],...,13,15,False,False,False,False,0.0,4.000000,0.032258,0.032468
249947,compétition compétition compétition compétitio...,0,0,393,10628,3015,[],['https://t.co/njojlsipoq'],0,[],...,17,19,False,False,False,False,0.0,11.000000,0.047619,0.035857
230653,poutine zemmour poutine zemmour poutine zemmou...,0,0,368,7542,734,[],['https://t.co/oigbtye60x'],0,[],...,16,18,False,True,False,False,0.0,7.000000,0.057143,0.039427
343100,francais merde francais merde francais merde f...,4,20,454,22221,649,[],['https://t.co/goonqmya9o'],0,[],...,10,8,False,False,False,False,0.0,7.923077,0.153846,0.047619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222510,demain,3,0,5674,118641,4333,[],[],0,[],...,19,20,False,False,False,False,0.0,6.000000,1.000000,1.000000
286152,studio,13,26,9078,68148,907,[],[],0,[],...,8,19,False,False,False,False,0.0,6.000000,1.000000,1.000000
304645,vote bami,0,0,597,4599,346,[],[],0,[],...,16,7,False,False,False,False,0.0,4.000000,1.000000,1.000000
46415,symbole,0,0,2706,21051,2026,[],[],0,[],...,17,7,False,False,False,False,0.0,7.000000,1.000000,1.000000


In [23]:
train_data.sort_values("rep_words_freq")

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,day,hour,Macron,Zemmour,Melenchon,rt,compound,avg_word_len,rep_words_freq,rep_chars_freq
134183,vote vote vote vote vote vote vote vote vote v...,0,0,164,31083,80,[],['https://t.co/04wft89icf'],0,[],...,18,21,False,False,False,False,0.0000,4.000000,0.025000,0.025126
102001,vote vote vote vote vote vote vote vote vote v...,9,11,363,5151,333,[],['https://t.co/w88mzuszit'],0,[],...,13,15,False,False,False,False,0.0000,4.000000,0.032258,0.032468
249947,compétition compétition compétition compétitio...,0,0,393,10628,3015,[],['https://t.co/njojlsipoq'],0,[],...,17,19,False,False,False,False,0.0000,11.000000,0.047619,0.035857
230653,poutine zemmour poutine zemmour poutine zemmou...,0,0,368,7542,734,[],['https://t.co/oigbtye60x'],0,[],...,16,18,False,True,False,False,0.0000,7.000000,0.057143,0.039427
192797,voten voten voten voten voten voten voten vote...,5,2,1028,21305,1054,[],[],0,[],...,17,4,False,False,False,False,0.0000,5.000000,0.058824,0.059406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130450,excellent moment actualité cinématographique j...,0,0,220,29232,73,[],['https://t.co/e4ruljrnb0'],0,[],...,11,6,False,False,False,False,0.5719,7.250000,1.000000,0.255102
130449,rt macron mondialistes,1,0,1737,33991,3202,[],[],0,[],...,12,19,True,False,False,True,0.0000,6.666667,1.000000,0.590909
130448,rt considèrent poutou prolétaire patrimoine pr...,1,0,809,172767,750,[],[],0,[],...,10,23,False,False,False,True,0.0000,7.285714,1.000000,0.298246
130457,mettre replay croyait zemmour femmes intellige...,38,81,987,2371,1819,[],['https://t.co/y6wnqxd1v7'],0,[],...,13,16,False,True,False,False,0.0000,6.555556,1.000000,0.298507


In [27]:
train_data["max_char_freq"] = train_data["text"].apply(lambda s: max( [s.count(c) for c in list(set(s))] )   /len(list(s)))

In [28]:
train_data.sort_values("max_char_freq")

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,hour,Macron,Zemmour,Melenchon,rt,compound,avg_word_len,rep_words_freq,rep_chars_freq,max_char_freq
249344,hugolaz président,0,0,677,14630,168,[],[],0,[],...,16,False,False,False,False,0.0,8.0,1.0,1.000000,0.058824
195240,jaloux président,0,0,5,129,68,[],[],0,[],...,11,False,False,False,False,0.0,7.5,1.0,1.000000,0.062500
133113,nb démocratique,1,0,523,38770,471,[],[],0,[],...,16,False,False,False,False,0.0,7.0,1.0,1.000000,0.066667
51437,intouchable fdp,0,0,40,2274,133,[],[],0,[],...,11,False,False,False,False,0.0,7.0,1.0,1.000000,0.066667
46940,garçon stupide,0,0,3,43,22,[],[],0,[],...,16,False,False,False,False,0.0,6.5,1.0,1.000000,0.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80828,eeeeeeeeh,0,0,134,11593,198,[],[],0,[],...,9,False,False,False,False,0.0,9.0,1.0,0.222222,0.888889
265106,ptdrrrrrrrrrrrrrrrrrrrrrrrrr,0,0,207,55628,190,[],[],0,[],...,19,False,False,False,False,0.0,28.0,1.0,0.142857,0.892857
78763,naaaaaaaaa,2,0,1399,28314,3360,[],[],0,['jadotprésident'],...,11,False,False,False,False,0.0,10.0,1.0,0.200000,0.900000
14154,ptdrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr...,0,0,818,12410,113,[],[],0,[],...,18,False,False,False,False,0.0,50.0,1.0,0.080000,0.940000


In [29]:
train_data["avg_word_count"] = train_data["text"].apply(lambda s: len(s.split()))

In [30]:
train_data.sort_values("avg_word_count")


,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,Macron,Zemmour,Melenchon,rt,compound,avg_word_len,rep_words_freq,rep_chars_freq,max_char_freq,avg_word_count
75256,médité,0,0,122,1755,284,[],[],0,[],...,False,False,False,False,0.0000,6.000000,1.000000,0.833333,0.333333,1
300105,xptdrrrrr,0,0,49,5232,43,[],[],0,[],...,False,False,False,False,0.0000,9.000000,1.000000,0.555556,0.555556,1
69237,centaines,2,0,1374,317529,521,[],['https://t.co/9tlkbair6d'],0,[],...,False,False,False,False,0.0000,9.000000,1.000000,0.777778,0.222222,1
320205,chronique,14,11,5161,8116,5286,[],[],0,['droite_alstom'],...,False,False,False,False,0.0000,9.000000,1.000000,1.000000,0.111111,1
98660,savait,1,0,23221,255094,15788,[],[],0,[],...,False,False,False,False,0.0000,6.000000,1.000000,0.833333,0.333333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194840,today the ukrainian people are defending not o...,0,0,15494,26751,5325,[],['https://t.co/hlakillyfp'],0,[],...,False,False,False,False,0.0516,5.270270,0.837838,0.112554,0.155844,37
77030,radio france devient france radio france devie...,3,0,430,33780,528,[],['https://t.co/uvuaznoc7m'],0,[],...,True,False,False,False,0.0000,6.000000,0.351351,0.089147,0.139535,37
308187,oui faut aller voter voter voter voter voter v...,2,0,798,6772,880,[],['https://t.co/3qd4thh8do'],0,[],...,False,False,False,False,0.0000,5.216216,0.243243,0.074236,0.157205,37
277985,domestic it been only an hour since the unoffi...,0,0,2,12,2,[],['https://t.co/mfglmnmezb'],0,[],...,False,False,False,False,0.4019,5.540541,0.891892,0.095436,0.149378,37
